### env

python

In [1]:
import sys

print(sys.version)

3.10.19 (main, Oct 21 2025, 16:43:05) [GCC 11.2.0]


CPU/GPU

In [2]:
import torch

# CPU/GPU 사용 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("현재 실행 장치: GPU (CUDA)")
    print("GPU 이름:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("현재 실행 장치: CPU")

현재 실행 장치: GPU (CUDA)
GPU 이름: NVIDIA RTX 6000 Ada Generation


### utils

In [3]:
import os
import pandas as pd
import ee
import re
from datetime import timedelta
import numpy as np

/home/khs/anaconda3/envs/hoso/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


### io

chl-a

In [4]:
# 최상위 폴더 경로
folder = r"/home/khs/data/csv/chla data/"

# 결과 저장
df_list = []

# 모든 하위 폴더 포함하여 .csv 찾기
for root, dirs, files in os.walk(folder):
    for file in files:
        if file.lower().endswith('.csv'):
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path, encoding= 'CP949')  # 또는 utf-8-sig
                df_list.append(df)
            except Exception as e:
                print(f"[오류] {file_path} 읽기 실패: {e}")

# 결측값 제거 후 전체 병합
if df_list:
    total_df = pd.concat(df_list, ignore_index=True)
    print(f"\n 총 병합된 파일 수: {len(df_list)}개")
    print(f" 총 병합된 행 수: {len(total_df)} rows")
    display(total_df.head())
else:
    print(" 병합할 수 있는 CSV 파일이 없습니다.")


 총 병합된 파일 수: 8개
 총 병합된 행 수: 52300 rows


,분류번호,측정소명,년/월/일,회차,경도,위도,채수시각,수심(m),수온(℃),클로로필 a(㎎/㎥),투명도(m),유량(㎥/s)
0,3303B30,부안댐1,2018/01/02,1회차,"126°33'35.2""","35°40'37.7""",10:02,15.0,4.9,1.7,3.0,NaN
1,1016B10,팔당댐5,2018/01/02,1회차,"127°17'47.58""","37°29'26.12""",12:00,NaN,NaN,NaN,NaN,NaN
2,3008B40,대청댐1,2018/01/02,1회차 상층부,"127°29'44""","36°22'16""",14:55,NaN,7.1,6.6,3.7,NaN
3,4007B60,동복댐1,2018/01/02,1회차 상층부,"127°6'4.12""","35°5'1.53""",NaN,NaN,6.5,3.7,3.0,NaN
4,4007B50,동복댐2,2018/01/02,1회차 상층부,"127°5'55.64""","35°5'32.28""",NaN,NaN,6.4,5.8,2.8,NaN


dam list

In [5]:
# 측정소명에서 숫자 + 괄호 제거 → 댐 이름만 추출
total_df['댐명'] = (
    total_df['측정소명']
    .str.replace(r'\d+', '', regex=True)        # 숫자 제거
    .str.replace(r'\(.*?\)', '', regex=True)    # 괄호 내용 제거
    .str.strip()                                # 공백 제거
)

# 중복 없는 댐 목록 추출
dam_list = total_df['댐명'].unique()

print(dam_list)

['부안댐' '팔당댐' '대청댐' '동복댐' '섬진강댐' '안계댐' '영천댐' '용담댐' '운문댐' '의암댐' '임하댐' '주암댐'
 '주암조정지댐' '충주댐' '경포호' '광동댐' '달방댐' '밀양댐' '소양강댐' '장흥댐' '평림댐' '원천지' '감포댐'
 '구천댐' '수어댐' '안동댐' '연초댐' '영산호' '장성댐' '청평댐' '충주조정지댐' '평화의댐' '남양호' '나주댐'
 '보령댐' '가창댐' '서호' '신갈지' '남강댐' '횡성댐' '고삼지' '예당지' '이동지' '주남저수지' '대암댐' '사연댐'
 '선암댐' '매호' '영랑호' '청초호' '향호' '군위댐' '대곡댐' '광포호' '봉포호' '송지호' '천진호' '화진포호'
 '화천댐' '보문호' '합천댐' '대아지' '대호' '광주댐' '낙동강하구' '광교지' '삽교호' '아산호' '동화호' '춘천댐'
 '금호호' '영암호' '경천지' '금강하구' '보성강댐' '회야호' '담양댐' '간월호' '괴산댐' '부남호' '탑정지' '영주댐'
 '김천부항댐' '보현산댐' '성덕댐' '한탄강댐' '군남댐' '도암댐']


water body list

In [6]:
# ---------------------
# 대형 댐호 (dam)
# ---------------------
dam_list = [
    '대청댐1','대청댐2','대청댐3','대청댐4','대청댐5(대청호)','대청댐6',
    '섬진강댐1(옥정호)','섬진강댐2(옥정호)','섬진강댐3(옥정호)',
    '임하댐1','임하댐2','임하댐3',

    '보령댐1','보령댐2','보령댐3',
    '부안댐1','부안댐2','부안댐3',
    '용담댐1','용담댐2','용담댐3','용담댐4',
    '장성댐1','장성댐2',

    '충주댐1','충주댐2','충주댐3','충주댐4',
    '수어댐1','수어댐2',
    '안계댐',

    '안동댐1','안동댐2','안동댐3',

    '영천댐1(영천호)','영천댐2(영천호)',
    '운문댐1','운문댐2',

    '의암댐1','의암댐2','의암댐3',

    '팔당댐1','팔당댐2','팔당댐3','팔당댐4','팔당댐5',

    '소양강댐1','소양강댐2','소양강댐3','소양강댐4','소양강댐5',

    '주암댐1','주암댐2','주암댐3',

    '군위댐1','군위댐2',
    '밀양댐1','밀양댐2',

    '횡성댐1','횡성댐2','횡성댐3',

    '광동댐','김천부항댐','달방댐',

    '나주댐1','나주댐2',
    '가창댐1','가창댐2',
    '구천댐',

    '남강댐1(진양호)','남강댐2(진양호)','남강댐3(진양호)',

    '연초댐1','연초댐2',

    '영주댐1','영주댐2','영주댐3','영주댐4',

    '청평댐1','청평댐2','청평댐3',

    '대곡댐1','대곡댐2',
    '대암댐1','대암댐2',
    '사연댐1','사연댐2',

    '선암댐',

    '장흥댐1','장흥댐2','장흥댐3','장흥댐4',

    '합천댐1','합천댐2','합천댐3',

    '괴산댐1','괴산댐2','괴산댐3',

    '담양댐1','담양댐2',

    '보성강댐1','보성강댐2',

    '화천댐1(파로호)','화천댐2(파로호)','화천댐3(파로호)',

    '춘천댐1','춘천댐2','춘천댐3',

    '한탄강댐','도암댐','군남댐','보현산댐','감포댐',

    '광주댐1','광주댐2',
    '회야호1','회야호2'
]


# ---------------------
# 조정지댐 (regulating)
# ---------------------
regulating_list = [
    '충주조정지댐1','충주조정지댐2',
    '주암조정지댐1(상사호)','주암조정지댐3(상사호)'
]


# ---------------------
# 자연호 (natural)
# ---------------------
natural_list = [
    '광포호','봉포호','송지호','천진호','화진포호',
    '영랑호','경포호1','경포호2','청초호',
    '서호1','서호2','서호3'
]


# ---------------------
# 인공저수지 (artificial)
# ---------------------
artificial_list = [
    '예당지1','예당지2','예당지3',
    '고삼지1','고삼지2','고삼지3',

    '경천지1','경천지2',
    '대아지1','대아지2','대아지3',

    '이동지1','이동지2',

    '향호','매호',

    '평림댐',

    '금호호1','금호호2','금호호3',

    '주남저수지',

    '보문호1','보문호2',

    '원천지1','원천지2','원천지3',
    '광교지1','광교지2',

    '아산호1(평택호)','아산호2(평택호)','아산호3(평택호)',

    '탑정지1(논산지)','탑정지2(논산지)',

    '동화호'
]


# ---------------------
# 간척호 (lagoon)
# ---------------------
lagoon_list = [
    '영암호1','영암호2','영암호3',
    '부남호1','부남호2','부남호3',
    '간월호1','간월호2','간월호3',
    '남양호1','남양호2','남양호3',
    '대호1','대호2','대호3',
    '삽교호1','삽교호2','삽교호3'
]


# ---------------------
# 하구호 (estuary)
# ---------------------
estuary_list = [
    '금강하구1','금강하구2','금강하구3',
    '낙동강하구1','낙동강하구2','낙동강하구3',
    '영산호1','영산호2','영산호3'
]

water body type selection

In [7]:
# 호소 유형 분류 함수
def classify_lake(name):
    if name in dam_list:
        return 'dam'
    elif name in natural_list:
        return 'natural'
    elif name in regulating_list:
        return 'regulating'
    elif name in artificial_list:
        return 'artificial'
    elif name in lagoon_list:
        return 'lagoon'
    elif name in estuary_list:
        return 'estuary'
    else:
        return 'unknown'

# total_df에 새로운 컬럼 추가
total_df["호소유형"] = total_df["측정소명"].apply(classify_lake)

# 결과 확인
print(total_df["호소유형"].value_counts())

호소유형
dam           38210
artificial     5523
lagoon         2801
estuary        1829
unknown        1657
regulating     1140
natural        1140
Name: count, dtype: int64


In [8]:
selected_types = ["dam", "regulating"]
total_df = total_df[ total_df["호소유형"].isin(selected_types) ].copy()

In [9]:
print("최종 선택된 호소유형 분포:")
print(total_df["호소유형"].value_counts())
print("최종 선택된 측정소 개수:", total_df["측정소명"].nunique())

최종 선택된 호소유형 분포:
호소유형
dam           38210
regulating     1140
Name: count, dtype: int64
최종 선택된 측정소 개수: 121


In [10]:
# 측정소명에서 숫자 + 괄호 제거 → 댐 이름만 추출
total_df['댐명'] = (
    total_df['측정소명']
    .str.replace(r'\d+', '', regex=True)        # 숫자 제거
    .str.replace(r'\(.*?\)', '', regex=True)    # 괄호 내용 제거
    .str.strip()                                # 공백 제거
)

# 중복 없는 댐 목록 추출
dam_list = total_df['댐명'].unique()

print(dam_list)

['부안댐' '팔당댐' '대청댐' '섬진강댐' '안계댐' '영천댐' '용담댐' '운문댐' '의암댐' '임하댐' '주암댐'
 '주암조정지댐' '충주댐' '광동댐' '달방댐' '밀양댐' '소양강댐' '장흥댐' '감포댐' '구천댐' '수어댐' '안동댐'
 '연초댐' '장성댐' '청평댐' '충주조정지댐' '나주댐' '보령댐' '가창댐' '남강댐' '횡성댐' '대암댐' '사연댐'
 '선암댐' '군위댐' '대곡댐' '화천댐' '합천댐' '광주댐' '춘천댐' '보성강댐' '회야호' '담양댐' '괴산댐' '영주댐'
 '김천부항댐' '보현산댐' '한탄강댐' '군남댐' '도암댐']


In [11]:
print(len(total_df))
total_df

39350


,분류번호,측정소명,년/월/일,회차,경도,위도,채수시각,수심(m),수온(℃),클로로필 a(㎎/㎥),투명도(m),유량(㎥/s),댐명,호소유형
0,3303B30,부안댐1,2018/01/02,1회차,"126°33'35.2""","35°40'37.7""",10:02,15.0,4.9,1.7,3.0,NaN,부안댐,dam
1,1016B10,팔당댐5,2018/01/02,1회차,"127°17'47.58""","37°29'26.12""",12:00,NaN,NaN,NaN,NaN,NaN,팔당댐,dam
2,3008B40,대청댐1,2018/01/02,1회차 상층부,"127°29'44""","36°22'16""",14:55,NaN,7.1,6.6,3.7,NaN,대청댐,dam
5,3303B20,부안댐2,2018/01/02,1회차 상층부,"126°34'1.35""","35°39'54.7""",10:00,28.8,3.0,1.9,2.5,NaN,부안댐,dam
6,3303B10,부안댐3,2018/01/02,1회차 상층부,"126°35'43.64""","35°38'54.59""",11:00,28.8,3.0,1.1,NaN,NaN,부안댐,dam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52295,1003B40,충주댐1,2019/12/30,5회차 하층부,"127°59'47.39""","37°0'2.52""",14:09,NaN,8.5,0.8,NaN,NaN,충주댐,dam
52296,1007B10,팔당댐1,2019/12/30,5회차 하층부,"127°25'46.62""","37°30'27.72""",14:58,NaN,4.1,2.9,NaN,NaN,팔당댐,dam
52297,1017B10,팔당댐2,2019/12/30,5회차 하층부,"127°17'.02""","37°31'17.02""",10:13,NaN,4.3,5.4,NaN,NaN,팔당댐,dam
52298,1007B20,팔당댐3,2019/12/30,5회차 하층부,"127°22'3.48""","37°31'35.14""",15:28,NaN,4.0,3.8,NaN,NaN,팔당댐,dam


In [12]:
total_df.count()

분류번호           39350
측정소명           39350
년/월/일          39350
회차             39350
경도             39350
위도             39350
채수시각           39092
수심(m)           7985
수온(℃)          36680
클로로필 a(㎎/㎥)    36640
투명도(m)         12645
유량(㎥/s)            0
댐명             39350
호소유형           39350
dtype: int64

In [13]:
total_df = total_df.dropna(subset=["클로로필 a(㎎/㎥)"])

In [14]:
print(len(total_df))

36640


### satellite

GEE

In [15]:
# 서버 사용 시 직접 터미널로 인증 진행

#기존 인증 파일 삭제(꼬였을 수 있으니 정리)
# rm ~/.config/earthengine/credentials

# 터미널에서 인증 실행
# earthengine authenticate --auth_mode=notebook

# 내 컴퓨터 사용 시
#ee.Authenticate()

In [17]:
# 최초 1회만 필요 (브라우저 인증)
ee.Initialize()

ImageCollection

In [18]:
# sentinel-2
S2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# 결과 저장용 리스트
results = []

matching

In [19]:
 ## 위치 단위 변환 필요
 # DMS->DD (이미 DD면 통과)
def dms_to_dd(v):
    try:
        if isinstance(v, (int, float)):
            return float(v)
        s = str(v).strip()
        if re.match(r'^\d+(\.\d+)?$', s):
            return float(s)
        parts = re.split('[°\'"]+', s)
        parts = [p for p in parts if p]
        deg, minute, sec = map(float, parts[:3])
        return deg + minute/60 + sec/3600
    except Exception:
        return None

sentinel-2

In [20]:
required_bands = [
    "SCL",
    "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B11", "B12"]
PATCH_SIZE = 3      # 3x3 패치
SCALE = 10          # Sentinel-2 해상도
HALF = PATCH_SIZE // 2

In [21]:
# -----------------------------------
# 설정
# -----------------------------------
PATCH_SIZE = 3
SCALE = 10
HALF = PATCH_SIZE // 2  # = 1

required_bands = [
    "SCL",
    "B1", "B2", "B3", "B4", "B5", "B6", "B7",
    "B8", "B8A", "B9", "B11", "B12"
]

kernel = ee.Kernel.square(radius=HALF)

results = []

print(f"시작: 총 {len(total_df['년/월/일'].unique())}개의 날짜 그룹을 처리합니다.\n")

# -----------------------------------
# 날짜별 처리
# -----------------------------------
for date, group in total_df.groupby("년/월/일"):
    obs_date = pd.to_datetime(date)
    start = obs_date.strftime("%Y-%m-%d")
    end   = (obs_date + timedelta(days=1)).strftime("%Y-%m-%d")

    print(f"[{date}] 처리 중...", end=" ")

    # -----------------------------------
    # FeatureCollection 생성
    # -----------------------------------
    features = []
    for idx, row in group.iterrows():
        lon = dms_to_dd(row["경도"])
        lat = dms_to_dd(row["위도"])
        features.append(
            ee.Feature(
                ee.Geometry.Point([lon, lat]),
                {"row_idx": int(idx)}
            )
        )
    fc = ee.FeatureCollection(features)

    # -----------------------------------
    # Sentinel-2 필터링
    # -----------------------------------
    s2_col = (
        S2
        .filterBounds(fc)
        .filterDate(start, end)
    )

    if s2_col.size().getInfo() == 0:
        print("→ [건너뜀] Sentinel-2 이미지 없음")
        continue

    s2_img = s2_col.sort("system:time_start", False).first()

    band_names = s2_img.bandNames().getInfo()
    if not all(b in band_names for b in required_bands):
        print("→ [건너뜀] 필수 밴드 누락")
        continue

    # -----------------------------------
    # SCL 마스킹
    # -----------------------------------
    scl = s2_img.select("SCL")
    mask = scl.eq(2).Or(scl.eq(6)).Or(scl.eq(7))

    s2_selected = s2_img.select(required_bands).updateMask(mask)

    image_id = s2_img.id().getInfo()

    # -----------------------------------
    #  3×3 패치 이미지 생성
    # -----------------------------------
    patch_img = s2_selected.neighborhoodToArray(kernel)

    # -----------------------------------
    #  포인트 기준 일괄 샘플링
    # -----------------------------------
    samples = patch_img.sampleRegions(
        collection=fc,
        scale=SCALE,
        geometries=False
    ).getInfo()

    success_count = 0

    for f in samples["features"]:
        props = f["properties"]
        idx = props["row_idx"]

        out = total_df.loc[idx].to_dict()

        # 밴드별 (3,3) 패치 복원
        for b in required_bands:
            if b not in props:
                break
            out[b] = np.array(props[b])  # (3,3)

        out["image_id_S2"] = image_id
        results.append(out)
        success_count += 1

    print(f"→ [완료] {success_count}개 지점 패치 추출 성공")

print("\n--- 모든 처리 완료 ---")

# -----------------------------------
# 최종 DataFrame
# -----------------------------------
if results:
    df_out = pd.DataFrame(results)
    print(f"최종 데이터 개수: {len(df_out)}개")
else:
    print("추출된 데이터가 없습니다.")

시작: 총 1319개의 날짜 그룹을 처리합니다.

[2018/01/02] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/03] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/04] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/05] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/08] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/09] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/10] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/12] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/15] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/19] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/22] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/23] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/01/29] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/02/01] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/02/02] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/02/05] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/02/06] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/02/08] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/02/09] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/02/12] 처리 중... → [건너뜀] Sentinel-2 이미지 없음
[2018/02/14] 처리 중... → [건너뜀] Sen

In [24]:
print(f"매칭 데이터 개수: {len(df_out)}개")
df_out

매칭 데이터 개수: 881개


,분류번호,측정소명,년/월/일,회차,경도,위도,채수시각,수심(m),수온(℃),클로로필 a(㎎/㎥),...,B4,B5,B6,B7,B8,B8A,B9,B11,B12,image_id_S2
0,5003B20,나주댐1,2018/02/22,1회차 상층부,"126°51'36""","34°57'22.39""",11:07,NaN,2.9,9.3,...,"[[256, 245, 253], [247, 245, 257], [244, 256, ...","[[238, 243, 243], [232, 239, 239], [232, 239, ...","[[227, 226, 226], [229, 225, 225], [229, 225, ...","[[233, 233, 233], [234, 243, 243], [234, 243, ...","[[249, 249, 253], [252, 249, 248], [247, 249, ...","[[221, 219, 219], [222, 220, 220], [222, 220, ...","[[195, 195, 195], [195, 195, 195], [195, 195, ...","[[180, 183, 183], [177, 188, 188], [177, 188, ...","[[146, 144, 144], [143, 139, 139], [143, 139, ...",20180222T021709_20180222T021703_T52SBD
1,5003B10,나주댐2,2018/02/22,1회차 상층부,"126°50'55.32""","34°55'39.36""",11:23,NaN,3.7,8.3,...,"[[650, 464, 458], [732, 511, 482], [0, 571, 507]]","[[866, 629, 629], [866, 629, 629], [0, 745, 745]]","[[824, 590, 590], [824, 590, 590], [0, 792, 792]]","[[918, 628, 628], [918, 628, 628], [0, 823, 823]]","[[882, 618, 542], [1100, 712, 618], [0, 802, 6...","[[1015, 556, 556], [1015, 556, 556], [0, 827, ...","[[473, 473, 473], [473, 473, 473], [0, 1337, 1...","[[880, 375, 375], [880, 375, 375], [0, 833, 833]]","[[667, 280, 280], [667, 280, 280], [0, 594, 594]]",20180222T021709_20180222T021703_T52SBD
2,5003B20,나주댐1,2018/02/22,1회차 중층부,"126°51'36""","34°57'22.39""",11:15,NaN,3.6,8.6,...,"[[256, 245, 253], [247, 245, 257], [244, 256, ...","[[238, 243, 243], [232, 239, 239], [232, 239, ...","[[227, 226, 226], [229, 225, 225], [229, 225, ...","[[233, 233, 233], [234, 243, 243], [234, 243, ...","[[249, 249, 253], [252, 249, 248], [247, 249, ...","[[221, 219, 219], [222, 220, 220], [222, 220, ...","[[195, 195, 195], [195, 195, 195], [195, 195, ...","[[180, 183, 183], [177, 188, 188], [177, 188, ...","[[146, 144, 144], [143, 139, 139], [143, 139, ...",20180222T021709_20180222T021703_T52SBD
3,5003B20,나주댐1,2018/02/22,1회차 하층부,"126°51'36""","34°57'22.39""",11:11,NaN,3.6,8.2,...,"[[256, 245, 253], [247, 245, 257], [244, 256, ...","[[238, 243, 243], [232, 239, 239], [232, 239, ...","[[227, 226, 226], [229, 225, 225], [229, 225, ...","[[233, 233, 233], [234, 243, 243], [234, 243, ...","[[249, 249, 253], [252, 249, 248], [247, 249, ...","[[221, 219, 219], [222, 220, 220], [222, 220, ...","[[195, 195, 195], [195, 195, 195], [195, 195, ...","[[180, 183, 183], [177, 188, 188], [177, 188, ...","[[146, 144, 144], [143, 139, 139], [143, 139, ...",20180222T021709_20180222T021703_T52SBD
4,2018B30,남강댐1(진양호),2018/10/15,1회차 상층부,"128°1'53.9""","35°10'4.8""",09:30,40.98,17.7,4.5,...,"[[854, 850, 835], [836, 832, 830], [844, 864, ...","[[645, 645, 644], [645, 645, 644], [645, 645, ...","[[172, 172, 178], [172, 172, 178], [179, 179, ...","[[154, 154, 154], [154, 154, 154], [155, 155, ...","[[137, 132, 128], [136, 125, 126], [134, 124, ...","[[79, 79, 77], [79, 79, 77], [75, 75, 84]]","[[0, 0, 0], [0, 0, 0], [0, 0, 0]]","[[28, 28, 29], [28, 28, 29], [28, 28, 27]]","[[36, 36, 38], [36, 36, 38], [33, 33, 39]]",20181015T021641_20181015T022428_T52SDD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,4007B70,주암댐1,2025/10/28,3회차 상층부,"127°14'26.74""","35°3'23.78""",10:10,NaN,21.2,6.0,...,"[[97, 72, 78], [90, 76, 75], [70, 60, 70]]","[[81, 73, 73], [78, 66, 66], [78, 66, 66]]","[[81, 62, 62], [68, 51, 51], [68, 51, 51]]","[[67, 70, 70], [52, 52, 52], [52, 52, 52]]","[[56, 51, 40], [56, 49, 43], [51, 47, 49]]","[[48, 61, 61], [51, 62, 62], [51, 62, 62]]","[[82, 82, 82], [69, 69, 69], [69, 69, 69]]","[[73, 69, 69], [72, 68, 68], [72, 68, 68]]","[[47, 44, 44], [45, 37, 37], [45, 37, 37]]",20251028T021821_20251028T022504_T52SCD
877,4007B70,주암댐1,2025/10/28,3회차 중층부,"127°14'26.74""","35°3'23.78""",10:10,NaN,20.0,5.8,...,"[[97, 72, 78], [90, 76, 75], [70, 60, 70]]","[[81, 73, 73], [78, 66, 66], [78, 66, 66]]","[[81, 62, 62], [68, 51, 51], [68, 51, 51]]","[[67, 70, 70], [52, 52, 52], [52, 52, 52]]","[[56, 51, 40], [56, 49, 43], [51, 47, 49]]","[[48, 61, 61], [51, 6

In [25]:
# DataFrame 생성
df_out = pd.DataFrame(results)

# NaN 값이 있는 행 제거 (B2 밴드에 값이 없는 경우 삭제)
# subset에 체크할 컬럼들을 넣으면, 그 컬럼들 중 하나라도 NaN이면 삭제합니다.
df_clean = df_out.dropna(subset=required_bands)

print(f"결측치 제거 데이터 개수: {len(df_clean)}개")
df_clean

결측치 제거 데이터 개수: 881개


,분류번호,측정소명,년/월/일,회차,경도,위도,채수시각,수심(m),수온(℃),클로로필 a(㎎/㎥),...,B4,B5,B6,B7,B8,B8A,B9,B11,B12,image_id_S2
0,5003B20,나주댐1,2018/02/22,1회차 상층부,"126°51'36""","34°57'22.39""",11:07,NaN,2.9,9.3,...,"[[256, 245, 253], [247, 245, 257], [244, 256, ...","[[238, 243, 243], [232, 239, 239], [232, 239, ...","[[227, 226, 226], [229, 225, 225], [229, 225, ...","[[233, 233, 233], [234, 243, 243], [234, 243, ...","[[249, 249, 253], [252, 249, 248], [247, 249, ...","[[221, 219, 219], [222, 220, 220], [222, 220, ...","[[195, 195, 195], [195, 195, 195], [195, 195, ...","[[180, 183, 183], [177, 188, 188], [177, 188, ...","[[146, 144, 144], [143, 139, 139], [143, 139, ...",20180222T021709_20180222T021703_T52SBD
1,5003B10,나주댐2,2018/02/22,1회차 상층부,"126°50'55.32""","34°55'39.36""",11:23,NaN,3.7,8.3,...,"[[650, 464, 458], [732, 511, 482], [0, 571, 507]]","[[866, 629, 629], [866, 629, 629], [0, 745, 745]]","[[824, 590, 590], [824, 590, 590], [0, 792, 792]]","[[918, 628, 628], [918, 628, 628], [0, 823, 823]]","[[882, 618, 542], [1100, 712, 618], [0, 802, 6...","[[1015, 556, 556], [1015, 556, 556], [0, 827, ...","[[473, 473, 473], [473, 473, 473], [0, 1337, 1...","[[880, 375, 375], [880, 375, 375], [0, 833, 833]]","[[667, 280, 280], [667, 280, 280], [0, 594, 594]]",20180222T021709_20180222T021703_T52SBD
2,5003B20,나주댐1,2018/02/22,1회차 중층부,"126°51'36""","34°57'22.39""",11:15,NaN,3.6,8.6,...,"[[256, 245, 253], [247, 245, 257], [244, 256, ...","[[238, 243, 243], [232, 239, 239], [232, 239, ...","[[227, 226, 226], [229, 225, 225], [229, 225, ...","[[233, 233, 233], [234, 243, 243], [234, 243, ...","[[249, 249, 253], [252, 249, 248], [247, 249, ...","[[221, 219, 219], [222, 220, 220], [222, 220, ...","[[195, 195, 195], [195, 195, 195], [195, 195, ...","[[180, 183, 183], [177, 188, 188], [177, 188, ...","[[146, 144, 144], [143, 139, 139], [143, 139, ...",20180222T021709_20180222T021703_T52SBD
3,5003B20,나주댐1,2018/02/22,1회차 하층부,"126°51'36""","34°57'22.39""",11:11,NaN,3.6,8.2,...,"[[256, 245, 253], [247, 245, 257], [244, 256, ...","[[238, 243, 243], [232, 239, 239], [232, 239, ...","[[227, 226, 226], [229, 225, 225], [229, 225, ...","[[233, 233, 233], [234, 243, 243], [234, 243, ...","[[249, 249, 253], [252, 249, 248], [247, 249, ...","[[221, 219, 219], [222, 220, 220], [222, 220, ...","[[195, 195, 195], [195, 195, 195], [195, 195, ...","[[180, 183, 183], [177, 188, 188], [177, 188, ...","[[146, 144, 144], [143, 139, 139], [143, 139, ...",20180222T021709_20180222T021703_T52SBD
4,2018B30,남강댐1(진양호),2018/10/15,1회차 상층부,"128°1'53.9""","35°10'4.8""",09:30,40.98,17.7,4.5,...,"[[854, 850, 835], [836, 832, 830], [844, 864, ...","[[645, 645, 644], [645, 645, 644], [645, 645, ...","[[172, 172, 178], [172, 172, 178], [179, 179, ...","[[154, 154, 154], [154, 154, 154], [155, 155, ...","[[137, 132, 128], [136, 125, 126], [134, 124, ...","[[79, 79, 77], [79, 79, 77], [75, 75, 84]]","[[0, 0, 0], [0, 0, 0], [0, 0, 0]]","[[28, 28, 29], [28, 28, 29], [28, 28, 27]]","[[36, 36, 38], [36, 36, 38], [33, 33, 39]]",20181015T021641_20181015T022428_T52SDD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,4007B70,주암댐1,2025/10/28,3회차 상층부,"127°14'26.74""","35°3'23.78""",10:10,NaN,21.2,6.0,...,"[[97, 72, 78], [90, 76, 75], [70, 60, 70]]","[[81, 73, 73], [78, 66, 66], [78, 66, 66]]","[[81, 62, 62], [68, 51, 51], [68, 51, 51]]","[[67, 70, 70], [52, 52, 52], [52, 52, 52]]","[[56, 51, 40], [56, 49, 43], [51, 47, 49]]","[[48, 61, 61], [51, 62, 62], [51, 62, 62]]","[[82, 82, 82], [69, 69, 69], [69, 69, 69]]","[[73, 69, 69], [72, 68, 68], [72, 68, 68]]","[[47, 44, 44], [45, 37, 37], [45, 37, 37]]",20251028T021821_20251028T022504_T52SCD
877,4007B70,주암댐1,2025/10/28,3회차 중층부,"127°14'26.74""","35°3'23.78""",10:10,NaN,20.0,5.8,...,"[[97, 72, 78], [90, 76, 75], [70, 60, 70]]","[[81, 73, 73], [78, 66, 66], [78, 66, 66]]","[[81, 62, 62], [68, 51, 51], [68, 51, 51]]","[[67, 70, 70], [52, 52, 52], [52, 52, 52]]","[[56, 51, 40], [56, 49, 43], [51, 47, 49]]","[[48, 61, 61], [51, 6

In [26]:
# 저장 경로 설정
save_path = r"/home/khs/data/hoso_data/"

# 파일명 결합 (전체 경로 생성)
output_filename = os.path.join(save_path, "chla_sentinel2_2D.csv")

# CSV 저장 (한글 깨짐 방지를 위해 utf-8-sig 사용)
df_clean.to_csv(output_filename, index=False, encoding='utf-8-sig')

# 파일 경로
npy_filename = os.path.join(save_path, "chla_sentinel2_2D_df.npy")

# DataFrame → numpy(object) 배열로 변환 후 저장
np.save(npy_filename, df_clean.to_numpy())

# 결과 출력
print(f"저장된 데이터 개수: {len(df_clean)}개")
print(f"csv 파일 저장 완료: {output_filename}")
print(f"NPY 파일 저장 완료: {npy_filename}")

NPY 파일 저장 완료: /home/khs/data/hoso_data/chla_sentinel2_2D_df.npy
저장된 데이터 개수: 881개
파일 저장 완료: /home/khs/data/hoso_data/chla_sentinel2_2D.csv


landsat

In [64]:
csv_path = "/home/khs/data/hoso_data/chla_sentinel2_2D.csv"
total_df = pd.read_csv(csv_path)

# 2. Landsat 8/9 Collection 설정
# Landsat 8 및 9 Collection 2 Level 2 (ST_B10 밴드 포함)
L8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
L9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
landsat_col = L8.merge(L9)

results = []

print(f"시작: 총 {len(total_df['년/월/일'].unique())}개의 날짜 그룹을 처리합니다.\n")

for date, group in total_df.groupby("년/월/일"):
    obs_date = pd.to_datetime(date)
    start = obs_date.strftime("%Y-%m-%d")
    end = (obs_date + timedelta(days=1)).strftime("%Y-%m-%d")

    print(f"[{date}] Landsat 매칭 중...", end=" ")

    # FeatureCollection 생성
    features = []
    for idx, row in group.iterrows():
        # dms_to_dd 함수가 이미 정의되어 있다고 가정합니다.
        lon = dms_to_dd(row["경도"])
        lat = dms_to_dd(row["위도"])
        features.append(ee.Feature(ee.Geometry.Point([lon, lat]), {"idx": int(idx)}))

    fc = ee.FeatureCollection(features)

    # 해당 날짜와 위치의 Landsat 이미지 필터링
    s_col = landsat_col.filterBounds(fc).filterDate(start, end)

    if s_col.size().getInfo() == 0:
        print("→ [건너뜀] 이미지 없음")
        continue

    # 최신 이미지 선택 및 수온 밴드(ST_B10) 추출
    # Scale Factor 적용: ST_B10 * 0.00341802 + 149.0 (Kelvin 단위)
    # 이후 -273.15를 더해 섭씨(Celsius)로 변환
    img = s_col.sort("system:time_start", False).first()

    temp_img = img.select("ST_B10").multiply(0.00341802).add(149.0).subtract(273.15)
    temp_img = temp_img.rename("Surface_Temp")

    try:
        # 데이터 추출
        reduced = temp_img.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.first(),
            scale=30 # Landsat Thermal 밴드는 30m 해상도
        )

        reduced_info = reduced.getInfo()
        image_id = img.id().getInfo()

        success_count = 0
        for feat in reduced_info["features"]:
            props = feat["properties"]
            if "Surface_Temp" not in props: continue

            idx = props["idx"]
            out = total_df.loc[idx].to_dict() # 기존 S2 데이터 포함
            out["L8_Surface_Temp"] = props["Surface_Temp"]
            out["image_id_L8"] = image_id

            results.append(out)
            success_count += 1

        print(f"→ [완료] {success_count}개 매칭")

    except Exception as e:
        print(f"→ [에러] {e}")

시작: 총 187개의 날짜 그룹을 처리합니다.

[2018/02/22] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2018/10/15] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2018/12/19] Landsat 매칭 중... → [완료] 0개 매칭
[2018/12/24] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/01/03] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/01/08] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/01/15] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/02/12] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/03/04] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/03/11] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/03/14] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/03/19] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/04/03] Landsat 매칭 중... → [완료] 0개 매칭
[2019/04/15] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/05/03] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/05/13] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/05/15] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/05/28] Landsat 매칭 중... → [완료] 0개 매칭
[2019/06/17] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/06/24] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/06/27] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/08/01] Landsat 매칭 중... → [건너뜀] 이미지 없음
[2019/08/08

In [65]:
print(len(results))

0
